In [226]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

grid definition

In [446]:
texp_dim=np.linspace(1,0,10)
spot_dim=np.arange(.05,2,.05)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

option definition

In [447]:
flag = 'c'  # 'c' for call, 'p' for put
K = 1  # Strikes
r=0
iv = 0.2  # Implied Volatilities
option_price=py_vollib.black_scholes.black_scholes(flag, spot_grid, K, texp_grid, r, iv,return_as='array')
option_greeks = py_vollib_vectorized.get_all_greeks(flag, 1, 1, .1, r, iv, return_as='dict')  

option_greeks['vega']


[0.0012609351106848202]

In [229]:
#90/110 risk reversal
cp =['p','c']
texp=[1,1]
r= 0
v = [0.2,0.2]  # Implied Volatilities
K = [0.9,1.1]  # Strikes
qty=[-100000000,100000000]
num_options=2

In [448]:
num_time_steps = 10
num_spots= 21
low_strike=.5   
high_strike=2

prepare for pricing

In [449]:
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

all_option_greeks={
    "delta":[],
    "gamma":[],
    "theta":[],
    "vega":[],
    "rho":[],
    "price":[]
}

for (qtyi,texpi,ki,cpi, ivi) in zip(qty,texp,K,cp,v):
    texp_gridi=texp_grid-(max_texp-texpi)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    option_price=py_vollib.black_scholes.black_scholes(cpi, spot_grid, ki, texp_gridi_zero, r, ivi,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(cpi, spot_grid, ki, texp_gridi_zero, r, ivi, return_as='dict')
    gr_names=list(option_greeks.keys())
    option_greeks_arr={}
    for gr in gr_names:
        arr=np.array(option_greeks[gr])
        arr=arr.reshape(len(texp_dim),len(spot_dim))
        if(gr=="gamma"):
            arr=scale_gamma(arr,1)/100*spot_grid/1000000
        option_greeks_arr[gr]=arr
        all_option_greeks[gr].append(arr)
    arr=np.array(option_price).reshape(len(texp_dim),len(spot_dim))
    #arr=option_price.reshape(len(texp_dim),len(spot_dim))
    all_option_greeks["price"].append(arr)

portfolio_greeks={}
for gr in list(all_option_greeks.keys()):
    for (qtyi,i) in zip(qty,range(len(qty))):
        if portfolio_greeks.get(gr) is None:
            portfolio_greeks[gr]=qtyi*all_option_greeks[gr][i]
        else:
            portfolio_greeks[gr]=portfolio_greeks[gr]+qtyi*all_option_greeks[gr][i]


#for (gr, qtyi) in zip(list(all_option_greeks.keys()),qty):
#    portfolio_greeks[gr]=np.sum(np.array(all_option_greeks[gr]*qtyi),axis=0)


TypeError: 'int' object is not iterable

In [153]:
def scale_gamma(arr, spot):
    return arr*spot*spot/100
    

In [450]:
#all_option_greeks.keys()
#portfolio_greeks["price"].shape
df = pd.DataFrame(portfolio_greeks["gamma"], columns = spot_dim, index = texp_dim)

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler


df.style.format('{:.2f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2)

,50.0%,57.5%,65.0%,72.5%,80.0%,87.5%,95.0%,102.5%,110.0%,117.5%,125.0%,132.5%,140.0%,147.5%,155.0%,162.5%,170.0%,177.5%,185.0%,192.5%,200.0%
2.00,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01,-0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1.80,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01,-0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1.60,-0.00,-0.00,-0.00,-0.01,-0.01,-0.01,-0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1.40,-0.00,-0.00,-0.00,-0.01,-0.02,-0.01,-0.00,0.01,0.01,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1.20,-0.00,-0.00,-0.00,-0.01,-0.02,-0.02,-0.01,0.01,0.02,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1.00,-0.00,-0.00,-0.00,-0.01,-0.02,-0.02,-0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.80,-0.00,-0.00,-0.00,-0.01,-0.02,-0.02,-0.01,0.01,0.02,0.02,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.60,-0.00,-0.00,-0.00,-0.01,-0.02,-0.03,-0.02,0.01,0.03,0.03,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.40,-0.00,-0.00,-0.00,-0.00,-0.02,-0.04,-0.02,0.02,0.04,0.03,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.20,0.00,-0.00,-0.00,-0.00,-0.01,-0.06,-0.04,0.03,0.06,0.04,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00


In [262]:
df.columns.format('{:.2f}')

['',
 '0.500',
 '0.575',
 '0.650',
 '0.725',
 '0.800',
 '0.875',
 '0.950',
 '1.025',
 '1.100',
 '1.175',
 '1.250',
 '1.325',
 '1.400',
 '1.475',
 '1.550',
 '1.625',
 '1.700',
 '1.775',
 '1.850',
 '1.925',
 '2.000']

In [451]:
def create_adjusted_texp_grid(texp_grid, option_texp, max_texp):
    texp_gridi=texp_grid-(max_texp-option_texp)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    return texp_gridi_zero

def create_constant_grid(param, num_time_steps, num_spots):
    cp_grid=np.full((num_time_steps,num_spots),param, dtype=type(param))
    return cp_grid
#create_constant_grid(35.0, num_time_steps, num_spots)


In [452]:
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

In [189]:
gr="delta"
np.array(option_greeks[gr]).shape
len(option_greeks[gr]

SyntaxError: incomplete input (4278250484.py, line 3)

In [453]:
pricing_grids=[]

for (qtyi,texpi,ki,cpi, ivi) in zip(qty,texp,K,cp,v):
    texp_gridi=create_adjusted_texp_grid(texp_grid, texpi, max_texp)
    cp_grid=create_constant_grid(cpi, num_time_steps, num_spots)
    iv_grid=create_constant_grid(ivi, num_time_steps, num_spots)
    ki_grid=create_constant_grid(ki, num_time_steps, num_spots)
    r_grid=create_constant_grid(r, num_time_steps, num_spots)
    qty_grid=create_constant_grid(qtyi, num_time_steps, num_spots)
    option_grids={
        "cp":cp_grid,
        "spot":spot_grid,
        "texp":texp_gridi,
        "iv":iv_grid,
        "ki":ki_grid,
        "r":r_grid,
        "qty":qty_grid
    }
    pricing_grids.append(option_grids)

# vstack the grids for each parameter into a single grid
inp_grids={}
for gr in list(pricing_grids[0].keys()):
    inp_grids[gr]=np.stack([pricing_grids[i][gr] for i in range(len(pricing_grids))])

option_price_c=py_vollib.black_scholes.black_scholes("c", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"],return_as='array')
option_greeks_c = py_vollib_vectorized.get_all_greeks("c", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"], return_as='dict')
option_price_p=py_vollib.black_scholes.black_scholes("p", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"],return_as='array')
option_greeks_p = py_vollib_vectorized.get_all_greeks("p", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"], return_as='dict')

option_greeks_c["price"]=option_price_c.reshape(num_options,num_time_steps,num_spots)
option_greeks_p["price"]=option_price_p.reshape(num_options,num_time_steps,num_spots)

for gr in list(option_greeks_c.keys()):
    option_greeks_c[gr]=np.array(option_greeks_c[gr]).reshape(num_options,num_time_steps,num_spots)
    option_greeks_p[gr]=np.array(option_greeks_p[gr]).reshape(num_options,num_time_steps,num_spots)


#option_price=np.where(inp_grids["cp"]=="c",option_price_c,option_price_p)
option_greeks={}
for gr in list(option_greeks_c.keys()):
    option_greeks[gr]=np.where(inp_grids["cp"]=="c",option_greeks_c[gr],option_greeks_p[gr])


for gr in list(option_greeks.keys()):
    option_greeks[gr]=np.array(option_greeks[gr]).reshape(num_options,num_time_steps,num_spots)


#option_greeks["price"]=option_price
option_greeks["qty"]=inp_grids["qty"]
option_greeks["gamma"]=scale_gamma(option_greeks["gamma"],inp_grids["spot"])

out_greeks={}
for gr in list(option_greeks.keys()):
    out_greeks[gr]=option_greeks[gr]*option_greeks["qty"]


out_greeks["gamma"]=out_greeks["gamma"]/1000000
out_greeks["vega"]=out_greeks["vega"]/1000

df = pd.DataFrame(out_greeks["gamma"].sum(axis=0), columns = spot_dim, index = texp_dim)
df.style.format('{:.1f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2)

TypeError: 'int' object is not iterable

In [ ]:
class option_pricing_date:
    
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

In [180]:
import ipysheet
sheet = ipysheet.sheet(rows=3, columns=4)
cell1 = ipysheet.cell(0, 0, 'Hello')
cell2 = ipysheet.cell(2, 0, 'World')
cell_value = ipysheet.cell(2,2, 42.)

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Hello'), Cell(colu…

Create option parameters from arrays and put them into a dataframe


In [503]:
qty=np.array([-1,1]*2)*1000000
texp=[1,1,2,2]
k=[.9,1.1]*2
k_dlr=k
cp=['p','c']*2
sigma=[.20,.2]*2


max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

In [504]:
data = {'qty': qty,
        'texp': texp,
        'k':k,
        '$k':k,
        'cp': cp,
        'sigma':sigma}
df_inp=pd.DataFrame(data)

In [505]:
def price_df_one_row(row):
    cp=row["cp"][0,0]
    spot=row["spot"]
    texp=row["texp"]
    r=row["r"]
    sigma=row["sigma"]
    k=row["k"]
    q=row["q"]
    spot_jump=spot*(1-.25)
    option_price=py_vollib_vectorized.models.vectorized_black_scholes_merton(cp, spot, k, texp, r, sigma,q,return_as='array')
    option_price_jump=py_vollib_vectorized.models.vectorized_black_scholes_merton(cp, spot_jump, k, texp, r, sigma,q,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(cp, spot, k, texp, r, sigma,q,model='black_scholes_merton', return_as='dict')
    option_greeks["price"]=option_price
    option_price_jump=np.array(option_price_jump).reshape(spot.shape)
    option_greeks = {k: np.array(v).reshape(spot.shape) for k, v in option_greeks.items()}
    option_greeks["jump_pl"]=(option_price_jump-option_greeks["price"])-option_greeks["delta"]*(spot_jump-spot)
    option_greeks = {k: np.where(texp>0,v,0) for k, v in option_greeks.items()}
    
    return(option_greeks)

In [508]:
r=0
q=0
spot_price=100

df_grids=df_inp.copy()

df_grids["r"]=r
df_grids["q"]=q
df_grids["texp"]=df_grids["texp"].apply(lambda x: create_adjusted_texp_grid(texp_grid, x, max_texp))
df_grids[["qty","k","cp","sigma"]]=df_inp[["qty","k","cp","sigma"]].applymap(lambda x: create_constant_grid(x, num_time_steps, num_spots))
df_grids[["q","r"]]=df_grids[["q","r"]].applymap(lambda x: create_constant_grid(x, num_time_steps, num_spots))
df_grids[["spot"]]=df_inp[["qty"]].applymap(lambda x: spot_grid)


gr=price_df_one_row(df_grids.iloc[0])
df_grids.iterrows
df_priced={ind: price_df_one_row(row) for (ind,row) in df_grids.iterrows()}
dict_grids={}
for field in df_priced[0].keys():
    dict_grids[field]=np.stack([el[field] for el in df_priced.values()])

dict_grids["qty"]=np.stack(list(df_grids["qty"]))
dict_grids["texp"]=np.stack(list(df_grids["texp"]))
dict_grids["gamma"]*=spot_grid/100
dict_sum_grids={}
for field in ["price","delta","gamma","vega","rho","theta","jump_pl"]:
    dict_sum_grids[field]=(dict_grids[field]*dict_grids["qty"]).sum(axis=0)

(dict_grids["delta"]*dict_grids["delta"]).sum(axis=0).shape



(10, 21)

In [536]:
import ipywidgets as widgets
from IPython.display import display
scales_factor={'price':1,'delta':1,'gamma':1/1000000*spot_price,'vega':1/1000*spot_price,'rho':spot_price,'theta':1,'jump_pl':spot_price/1000000}
show_fields=["price","delta","gamma","vega","rho","theta","jump_pl"]

w_greek_selector=widgets.Dropdown(
    options=list(dict_sum_grids.keys()),
    value=list(dict_sum_grids.keys())[3],
    #rows=10,
    description='Columns to graph',
    disabled=False
)
w_min_spot=widgets.FloatRangeSlider(
    value=[5,200],
    min=0.0,
    max=1000.0,
    step=0.5,
    description='Spot:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='0.1f'
)
w_time_steps=widgets.BoundedIntText(
    value=10,
    min=1,
    max=20.0,
    step=1,
    description='Time Step:',
    disabled=False
)

w_layout_box=widgets.VBox([w_greek_selector,w_min_spot,w_time_steps])
display(w_layout_box)
def display_field(field):
    df = pd.DataFrame(dict_sum_grids[field]*scales_factor[field], columns = spot_dim, index = texp_dim)
    display(df.style.format('{:.1f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2))
out1 = widgets.interactive_output(display_field, {'field': w_greek_selector})

display(out1)

Output()

In [539]:
dict_grids["delta"][3,:,:].shape
df = pd.DataFrame(dict_grids["price"][0,:,:], columns = spot_dim, index = texp_dim)
display(df.style.format('{:.3f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2))

,50.0%,57.5%,65.0%,72.5%,80.0%,87.5%,95.0%,102.5%,110.0%,117.5%,125.0%,132.5%,140.0%,147.5%,155.0%,162.5%,170.0%,177.5%,185.0%,192.5%,200.0%
2.00,0.400,0.326,0.253,0.186,0.129,0.084,0.051,0.030,0.016,0.009,0.004,0.002,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.80,0.400,0.325,0.252,0.183,0.123,0.077,0.044,0.023,0.012,0.005,0.002,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.60,0.400,0.325,0.251,0.180,0.117,0.068,0.036,0.017,0.007,0.003,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.40,0.400,0.325,0.250,0.177,0.110,0.058,0.026,0.010,0.003,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.20,0.400,0.325,0.250,0.175,0.103,0.046,0.014,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.80,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.60,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.40,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
0.20,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
